# Introducción a la Ciencia de Datos: Tarea 2

Este notebook contiene el código de base para realizar la Tarea 2 del curso. Puede copiarlo en su propio repositorio y trabajar sobre el mismo.
Las **instrucciones para ejecutar el notebook** están en la [página inicial del repositorio](https://gitlab.fing.edu.uy/maestria-cdaa/intro-cd/).

**Se espera que no sea necesario revisar el código para corregir la tarea**, ya que todos los resultados y análisis relevantes deberían estar en el **informe en formato PDF**.


## Cargar bibliotecas (dependencias)
Recuerde instalar los requerimientos (`requirements.txt`) en el mismo entorno donde está ejecutando este notebook (ver [README](https://gitlab.fing.edu.uy/maestria-cdaa/intro-cd/)). Para la entrega 2 hay nuevas dependencias, por lo que es importante correr la siguiente celda.

In [5]:
!pip install -r ../requirements.txt
!pip install spacy
!python -m spacy download es_core_news_sm

error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try brew install
    xyz, where xyz is the package you are trying to
    install.
    
    If you wish to install a Python library that isn't in Homebrew,
    use a virtual environment:
    
    python3 -m venv path/to/venv
    source path/to/venv/bin/activate
    python3 -m pip install xyz
    
    If you wish to install a Python application that isn't in Homebrew,
    it may be easiest to use 'pipx install xyz', which will manage a
    virtual environment for you. You can install pipx with
    
    brew install pipx
    
    You may restore the old behavior of pip by passing
    the '--break-system-packages' flag to pip, or by adding
    'break-system-packages = true' to your pip.conf file. The latter
    will permanently disable this error.
    
    If you disable this error, we STRONGLY recommend that you additionally
    pass the '--user' flag to pip, or set 

In [7]:
import re
import os

from time import time
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.corpus import wordnet
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab')  # Necesario para tokenizar
from nltk.stem import WordNetLemmatizer
nltk.download('omw-1.4')   # Para sinónimos y definiciones
nltk.download('averaged_perceptron_tagger_eng')  # Para etiquetas gramaticales

import matplotlib.patches as mpatches
from mpl_toolkits.mplot3d import Axes3D

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import PCA
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

!pip install wordcloud
from wordcloud import WordCloud

ModuleNotFoundError: No module named 'numpy'

## Lectura de Datos

In [8]:
# DataFrame con todos los discursos:
pd.set_option('display.max_rows', None)
#df_speeches = pd.read_csv('C:/Users/lulag/introCD/data/us_2020_election_speeches.csv')
df_speeches = pd.read_csv('../data/us_2020_election_speeches.csv')
df_speeches

NameError: name 'pd' is not defined

In [10]:
# Separación de speakers múltiples
df_speeches['speaker'] = df_speeches['speaker'].str.split(',')
df_speeches = df_speeches.explode('speaker')
df_speeches['speaker'] = df_speeches['speaker'].str.strip()

In [ ]:
# Selección de los 3 candidatos con más discursos
presidents = ["Joe Biden", "Donald Trump", "Mike Pence"]

# df_speeches_top_3 = ...
def presencia(df, column, names):
    return df[column].astype(str).apply(lambda x: 1 if any(name in x for name in names) else 0)
boolean = presencia(df_speeches, 'speaker', presidents)    
df_speeches_top_3 = df_speeches[boolean == 1]
# Eliminación de Donald Trump Jr. (hijo)
df_speeches_top_3 = df_speeches_top_3.drop(136)

# Reseteo del índice
df_speeches_top_3.reset_index(drop=True, inplace=True)
df_speeches_top_3

## Limpieza de Textos

In [12]:
# 1 - Función clean_text() de la entrega anterior

def clean_text(df, column_name):
    # Eliminar primeras palabras hasta el primer "\n"
    result = df[column_name].str.replace(r"^[^\n]*\n", "", regex=True)
    # Convertir todo a minúsculas
    result = result.str.lower()
    # Completar signos de puntuación faltantes
    for punc in ["\n", "[", "]", ",", ":", ".", ";", "!", "”", "“", "-", "/", "(", ")", "?","…","’","‘"]:
        result = result.str.replace(punc, " ")
    return result

# Creación de una nueva columna CleanText a partir de text
df_speeches_top_3["CleanText"] = clean_text(df_speeches_top_3,'text')
# df_speeches_top_3["CleanText"]

In [13]:
# 2 - Eliminación de stopwords

# Lista de stopwords
stop_words = set(stopwords.words('english'))
# print(stop_words)

# Función para eliminar stopwords de un texto
def remove_stopwords(text):
    tokens = word_tokenize(text)  # Divide el texto en palabras
    filtered = [word for word in tokens if word not in stop_words]
    return ' '.join(filtered)

df_speeches_top_3["CleanText"] = df_speeches_top_3["CleanText"].apply(remove_stopwords)
# df_speeches_top_3["CleanText"]

In [14]:
# 3 - Separación de contracciones y escritura de forma completa

# La lista de stopwords que se eliminaron en la celda anterior incluye varias contracciones.
# En este paso se expande la contracciones que puedan no haber sido consideradas por la lista de stopwords de nltk.
contraction_list = {
    "let's": "let us",
    "what's": "what is",
    "it's": "it is",
    "you'll": "you will",
    "i'll": "i will",
    "i'm": "i am",
    "he'll": "he will",
    "she'll": "she will",
    "they'll": "they will",
    "we'll": "we will",
    "can't": "cannot",
    "won't": "will not",
    "doesn't": "does not",
    "don't": "do not",
    "isn't": "is not",
    "aren't": "are not",
    "hasn't": "has not",
    "haven't": "have not",
    "hadn't": "had not",
    "i've": "i have",
    "you've": "you have",
    "you're": "you are",
    "we've": "we have",
    "we're": "we are",
    "they're": "they are",
    "they've": "they have",
    "he's": "he is",
    "she's": "she is",
    "it's": "it is",
    "that's": "that is",
    "there's": "there is",
    "here's": "here is",
    "wasn't": "was not",
    "weren't": "were not",
    "didn't": "did not",
    "wouldn't": "would not",
    "shouldn't": "should not",
    "couldn't": "could not",
    "mustn't": "must not",
    "mightn't": "might not",
    "needn't": "need not",
    "i'd": "i would",
    "you'd": "you would",
    "he'd": "he would",
    "she'd": "she would",
    "we'd": "we would",
    "they'd": "they would",
    "it'd": "it would",
    "who's": "who is",
    "who've": "who have",
    "who'd": "who would",
    "how's": "how is",
    "when's": "when is",
    "why's": "why is",
    "ain't": "is not",
}
# Sustitución del apóstrofe gráfico
df_speeches_top_3["CleanText"] = df_speeches_top_3["CleanText"].apply(lambda x: x.replace("’", "'") if isinstance(x, str) else x)
def expand_contractions(text, contractions=contraction_list):
    pattern = re.compile(r'\b(' + '|'.join(re.escape(k) for k in contractions.keys()) + r')\b')
    return pattern.sub(lambda x: contractions[x.group()], text)

df_speeches_top_3["CleanText"] = df_speeches_top_3["CleanText"].apply(expand_contractions)

In [15]:
# 4 - Eliminación de números

def remove_numbers(text):
    # Elimina números, 
    # incluyendo sufijos como: 'th', 'st', 'nd', 'rd', 's'.
    return re.sub(r'\b\d+(?:\.\d+)?(?:st|nd|rd|th|s|k|ks|census)?\b', '', text)

df_speeches_top_3["CleanText"] = df_speeches_top_3["CleanText"].apply(remove_numbers)

In [16]:
# 5 - Lematización

# Inicialización del lematizador
lemmatizer = WordNetLemmatizer()

# Función para mapear etiquetas POS de nltk a las que requiere WordNet
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # default

# Función de lematización
def lemmatize_text(text):
    tokens = word_tokenize(text)
    pos_tags = nltk.pos_tag(tokens)
    lemmatized = [lemmatizer.lemmatize(word, get_wordnet_pos(tag)) for word, tag in pos_tags]
    return ' '.join(lemmatized)

df_speeches_top_3["CleanText"] = df_speeches_top_3["CleanText"].apply(lemmatize_text)
# df_speeches_top_3["CleanText"]

## Parte 1: Dataset y representación numérica de texto

In [ ]:
# 1: Separar del 30% del conjunto para test. Al 70 % restante para entrenamiento se lo llama "dev" (desarrollo).

# Definición de variables
X = df_speeches_top_3["CleanText"]      # Características: transcripciones de discursos (columna 'CleanText' del DataFrame)
y = df_speeches_top_3["speaker"]        # Etiqueta: presidente que pronunció el discurso (columna 'speaker' del DataFrame)

# División estratificada
X_dev, X_test, y_dev, y_test = train_test_split(
    X, y,
    test_size = 0.3,            # 30% se separa para test
    stratify = y,               # Respeta las proporciones entre candidatos
    random_state = 17           # Para reproducibilidad
)

print(f"Tamaños de los conjuntos: \nEntrenamiento:{X_dev.shape} Testeo:{X_test.shape}")

In [ ]:
# 2: Visualización de la proporción de cada candidato por conjunto

# Creación de un DataFrame con las proporciones
train_dist = y_dev.value_counts(normalize=True).rename("Entrenamiento")
test_dist = y_test.value_counts(normalize=True).rename("Evaluación")
balance_df = pd.concat([train_dist, test_dist], axis=1)

# Gráfico de barras
balance_df.plot(kind='bar')
plt.title("Proporción de discursos por candidato \nen los conjuntos de entrenamiento y evaluación")
plt.xlabel("Candidato")
plt.ylabel("Proporción")
plt.xticks(rotation=0)
plt.ylim(0, 1)
plt.grid(axis='y')
plt.legend(title='Conjunto:')
plt.show()

In [ ]:
# 3: Transforme el texto del conjunto de entrenamiento a la representación numérica (features) de conteo de palabras o bag of words.

# Creación del vectorizador
vectorizer = CountVectorizer()

# Entrenamiento y transformación de los textos
X_dev_bow = vectorizer.fit_transform(X_dev)

# Mostrar la forma de la matriz
print("Dimensiones de la matriz BoW:", X_dev_bow.shape)
# Ejemplo: primeras 5 palabras del vocabulario
print("Primeras palabras del vocabulario:", vectorizer.get_feature_names_out()[10:20])

In [ ]:
# 4: Obtenga la representación numérica Term Frequency - Inverse Document Frequency.

# Inicialización del transformador
tfidf_transformer = TfidfTransformer()
# Ajuste y transformación de la matriz de conteo
X_dev_tfidf = tfidf_transformer.fit_transform(X_dev_bow)

# Mostrar la forma de la matriz
print("Dimensiones de la matriz TfIdf:", X_dev_tfidf.shape)

In [ ]:
# Visualización de TF-IDF > 0 para un discurso de ejemplo

# Para el primer speech: Joe Biden
text_index = 0

# Vector TF-IDF en formato denso
text_vector = X_dev_tfidf[text_index].toarray()[0]

# Nombres de las palabras
feature_names = vectorizer.get_feature_names_out()

# Filtro de términos con TF-IDF > 0
non_zero_terms = [(feature_names[i], text_vector[i]) 
                  for i in range(len(text_vector)) if text_vector[i] > 0]

# Visualización de términos con su peso (ordenados por importancia)
non_zero_terms_sorted = sorted(non_zero_terms, key=lambda x: x[1], reverse=True)
# Primeros 10
print(non_zero_terms_sorted[:10])

In [ ]:
# Visualización de lo anterior como Wordmap

# Creación del diccionario: {palabra: peso TF-IDF}
word_weights = {
    feature_names[i]: text_vector[i]
    for i in range(len(text_vector)) if text_vector[i] > 0
}

# Generación de la nube de palabras
wordcloud = WordCloud(width=800, height=400, background_color='white')
wordcloud.generate_from_frequencies(word_weights)
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title(f'Nube de palabras - Discurso #{text_index} {df_speeches_top_3.iloc[text_index]["speaker"]}')
plt.show()

In [ ]:
# 5 :Muestre en un mapa el conjunto de entrenamiento, utilizando las dos primeras componentes PCA sobre los vectores de tf-idf.

# Aplicación de PCA 
components = 2
pca = PCA(n_components = components)
X_pca = pca.fit_transform(X_dev_tfidf.toarray())

# Obtenención de etiquetas
labels = y_dev.values

# Scatter Plot
plt.figure(figsize=(10, 6))
for speaker in set(labels):
    idx = labels == speaker
    plt.scatter(X_pca[idx, 0], X_pca[idx, 1], label=speaker, alpha=0.7)

plt.xlabel("Componente Principal 1")
plt.ylabel("Componente Principal 2")
plt.title("Proyección PCA de los discursos (TF-IDF)")
plt.legend(title="Candidato:")
plt.grid(True)
plt.show()

In [ ]:
# ¿Qué palabras contribuyen más al Componente Principal 1?

# Obtenención de las componentes (matriz: n_componentes x n_palabras)
components = pca.components_

# Obtenención de nombres de las palabras del vectorizador
feature_names = vectorizer.get_feature_names_out()

# Para la primera componente
comp_idx = 0  
top_indices = np.argsort(components[comp_idx])[::-1]  # Mayor contribución primero

# Mostrar las 10 palabras que más contribuyen a la primera componente
top_words = [(feature_names[i], components[comp_idx][i]) for i in top_indices[:10]]
print("Palabras que más contribuyen a la componente principal 1:")
for word, weight in top_words:
    print(f"{word}: {weight:.4f}")

In [ ]:
# Selección de palabras que más contribuyen al Componente Principal 1
top_words = [(feature_names[i], components[comp_idx][i]) for i in top_indices[:100]]  # Se toma 100 para mejor visualización

# Creación de diccionario con pesos
word_weights = {word: abs(weight) for word, weight in top_words}  # abs para que no haya pesos negativos

# Generación de nube de palabras
wordcloud = WordCloud(width=800, height=400, background_color='white', colormap='tab10').generate_from_frequencies(word_weights)
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title("Palabras que más contribuyen a PCA1", fontsize=16)
plt.show()

In [ ]:
# Selección de 3 Componentes Principales

# PCA con 3 componentes
components = 3
pca_3d = PCA(n_components = components)
X_pca_3d = pca_3d.fit_transform(X_dev_tfidf.toarray())  # Pasa el array a denso si es sparse

# Figura 3D
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')

for speaker in set(y_dev):
    idx = y_dev == speaker
    ax.scatter(X_pca_3d[idx, 0], X_pca_3d[idx, 1], X_pca_3d[idx, 2], label=speaker, s=50, alpha=0.7)

ax.set_xlabel("Componente Principal 1")
ax.set_ylabel("Componente Principal 2")
ax.set_zlabel("Componente Principal 3", labelpad=0)
ax.set_title("Visualización 3D de PCA los discursos (TF-IDF)")
ax.legend(title="Candidato:")
plt.tight_layout(pad=2.5)
plt.show()

In [ ]:
# Haga una visualización que permita entender cómo varía la varianza explicada a medida que se agregan componentes (e.g: hasta 10 componentes).

# PCA sobre la matriz TF-IDF
components = 100
pca = PCA(n_components = components)
X_pca = pca.fit_transform(X_dev_tfidf.toarray())

# Varianza explicada individual
varianza_explicada = pca.explained_variance_ratio_
# Varianza explicada acumulada
varianza_acumulada = np.cumsum(varianza_explicada)

# Gráfico
plt.figure(figsize=(16, 5))
plt.plot(range(1, components+1), varianza_acumulada, marker='o', linestyle='--', color='b')
plt.xticks(range(1, components+1, 3))
plt.xlabel('Número de Componentes Principales')
plt.ylabel('Varianza Explicada Acumulada')
plt.title('Varianza Explicada Acumulada vs. Número de Componentes')
plt.show()

## Parte 2: Entrenamiento y Evaluación de Modelos


In [358]:
# 1: Entrene el modelo Multinomial Naive Bayes, luego utilícelo para predecir sobre el conjunto de test, y reporte el valor de accuracy y la matriz de confusión. Reporte el valor de precision y recall para cada candidato. 
# Calcular matriz de confusión Sugerencia: utilice el método from_predictions de ConfusionMatrixDisplay para realizar la matriz.

# Primero ajustar y transformar los datos de entrenamiento
X_dev_bow = vectorizer.fit_transform(X_dev)
X_dev_tfidf = tfidf_transformer.fit_transform(X_dev_bow)

# Luego transformar los datos de test usando el vectorizador y transformador ya ajustados
X_test_bow = vectorizer.transform(X_test)
X_test_tfidf = tfidf_transformer.transform(X_test_bow)

# Entrenar el modelo Multinomial Naive Bayes
mnb = MultinomialNB()
mnb.fit(X_dev_tfidf, y_dev)

# Realizar predicciones sobre el conjunto de test
y_pred = mnb.predict(X_test_tfidf)

# Calcular accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Calcular precision y recall para cada candidato
precision = precision_score(y_test, y_pred, average=None)
recall = recall_score(y_test, y_pred, average=None)

# Crear un DataFrame para mostrar las métricas por candidato
metrics_df = pd.DataFrame({
    'Precision': precision,
    'Recall': recall
}, index=mnb.classes_)
print("\nMétricas por candidato:")
print(metrics_df)

# Visualizar la matriz de confusión
plt.figure(figsize=(8, 6))
ConfusionMatrixDisplay.from_predictions(
    y_test, 
    y_pred,
    display_labels=mnb.classes_,
    cmap='Blues'
)
plt.title('Matriz de Confusión - Multinomial Naive Bayes')
plt.show()


In [359]:
/

In [360]:
# 3: Elija el mejor modelo (mejores parámetros) y vuelva a entrenar sobre todo el conjunto de entrenamiento disponible (sin quitar datos para validación). Reporte el valor final de las métricas y la matriz de confusión.


In [361]:
# 4: Evalúe con validación cruzada al menos un modelo más (dentro de scikit-learn) aparte de Multinomial Naive Bayes para clasificar el texto utilizando las mismas features de texto.


In [362]:
# 5: Evalúe el problema cambiando al menos un candidato. En particular, observe el (des)balance de datos y los problemas que pueda generar, así como cualquier indicio que pueda ver en el mapeo previo con PCA.

In [363]:
# OPCIONAL: Repetir la clasificación con los tres candidatos con más discursos, pero esta vez clasificando a nivel de párrafos y no de discursos enteros.
